# Dataset

In [3]:
import sys
sys.path.append('../../datasets/')
import pickle
import numpy as np
import matplotlib.pyplot as plt
import h5py
from prepare_individuals import prepare, germanBats

classes = germanBats

In [4]:
patch_len = 44                               # 88 bei 44100, 44 bei 22050 = 250ms ~ 25ms

In [6]:
X_train, Y_train, X_test, Y_test, X_val, Y_val = prepare("../datasets/prepared.h5", classes, patch_len=patch_len)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:03<00:00,  4.56it/s]


In [7]:
print("Total calls:", len(X_train) + len(X_test) + len(X_val))
print(X_train.shape, Y_train.shape)

Total calls: 53759
(31594, 44, 257) (31594,)


In [7]:
# inverse distribution sampling
grouped = [[] for x in range(len(classes))]
for x, y in zip(X_train, Y_train):
    grouped[y].append((x, y))
sorted_list = list(sorted(grouped, key=len)) # class with least elements first
sampling = []
size = 1.0
for group in sorted_list:
    take = int(len(group) * size)
    print(len(group), "take", take)
    samples = np.asarray(group)[range(take)]
    sampling.extend(samples[:, 0])
    size = size / 2
    
print(len(sampling))

72 take 72
278 take 139
439 take 109
476 take 59
566 take 35
1102 take 34
1204 take 18
1227 take 9
1300 take 5
1353 take 2
1368 take 1
1381 take 0
1415 take 0
1669 take 0
2396 take 0
2587 take 0
3053 take 0
4213 take 0
5773 take 0
483


In [11]:
X = np.asarray(sampling + list(X_train[:2000]))
Y = None
with open('../datasets/call_nocall.annotation', 'rb') as file:
    Y = pickle.load(file)
print(X.shape, Y.shape)

call = Y.sum()
no_call = Y.shape[0] - call
print("Calls:", call, "\tNo Calls:", no_call)

(2483, 44, 257) (2483,)
Calls: 1431 	No Calls: 1052


# Model

In [1]:
import time
import datetime
import numpy as np
import tqdm
import torch
import torch.nn as nn
import torchvision
from torch.cuda.amp import autocast
from torch.utils.data import TensorDataset, DataLoader
from timm.data.mixup import Mixup

In [13]:
use_adam = True
use_reduceonplateu = True
use_mixup = False

In [14]:
mixup_args = {
    'mixup_alpha': 1.,
    'cutmix_alpha': 0.,
    'cutmix_minmax': None,
    'prob': 1.0,
    'switch_prob': 0.,
    'mode': 'batch',
    'label_smoothing': 0,
    'num_classes': 2}
mixup_fn = Mixup(**mixup_args)

In [15]:
class Block(nn.Module):
    def __init__(self, num_layers, in_channels, out_channels, identity_downsample=None, stride=1):
        assert num_layers in [18, 34, 50, 101, 152], "should be a a valid architecture"
        super(Block, self).__init__()
        self.num_layers = num_layers
        if self.num_layers > 34:
            self.expansion = 4
        else:
            self.expansion = 1
            
        # ResNet50, 101, and 152 include additional layer of 1x1 kernels
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(out_channels)
        if self.num_layers > 34:
            self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        else:
            # for ResNet18 and 34, connect input directly to (3x3) kernel (skip first (1x1))
            self.conv2 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * self.expansion, kernel_size=1, stride=1, padding=0)
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample

    def forward(self, x):
        identity = x
        if self.num_layers > 34:
            x = self.conv1(x)
            x = self.bn1(x)
            x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        #x = torchvision.ops.stochastic_depth(input=x, p=0.1, mode='batch', training=self.training)  # randomly zero input tensor
        x += identity
        x = self.relu(x)
        return x

In [16]:
class ResNet(nn.Module):
    def __init__(self, num_layers, block, image_channels, num_classes):
        assert num_layers in [18, 34, 50, 101, 152], f'ResNet{num_layers}: Unknown architecture! Number of layers has ' \
                                                     f'to be 18, 34, 50, 101, or 152 '
        super(ResNet, self).__init__()
        if num_layers < 50:
            self.expansion = 1
        else:
            self.expansion = 4
        if num_layers == 18:
            layers = [2, 2, 2, 2]
        elif num_layers == 34 or num_layers == 50:
            layers = [3, 4, 6, 3]
        elif num_layers == 101:
            layers = [3, 4, 23, 3]
        else:
            layers = [3, 8, 36, 3]
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # ResNetLayers
        self.layer1 = self.make_layers(num_layers, block, layers[0], intermediate_channels=64, stride=1)
        self.layer2 = self.make_layers(num_layers, block, layers[1], intermediate_channels=128, stride=2)
        self.layer3 = self.make_layers(num_layers, block, layers[2], intermediate_channels=256, stride=2)
        self.layer4 = self.make_layers(num_layers, block, layers[3], intermediate_channels=512, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * self.expansion, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        return x

    def make_layers(self, num_layers, block, num_residual_blocks, intermediate_channels, stride):
        layers = []

        identity_downsample = nn.Sequential(nn.Conv2d(self.in_channels, intermediate_channels*self.expansion, kernel_size=1, stride=stride),
                                            nn.BatchNorm2d(intermediate_channels*self.expansion))
        layers.append(block(num_layers, self.in_channels, intermediate_channels, identity_downsample, stride))
        self.in_channels = intermediate_channels * self.expansion # 256
        for i in range(num_residual_blocks - 1):
            layers.append(block(num_layers, self.in_channels, intermediate_channels)) # 256 -> 64, 64*4 (256) again
        return nn.Sequential(*layers)

In [17]:
def train_epoch(model, epoch, criterion, optimizer, scheduler, dataloader, device):
    model.train()
    
    running_loss = 0.0
    running_corrects = 0
    
    num_batches = len(dataloader)
    num_samples = len(dataloader.dataset)
    
    for batch, (inputs, labels) in enumerate(tqdm.tqdm(dataloader)):
        # Transfer Data to GPU if available
        inputs, labels = inputs.to(device), labels.to(device)
        if use_mixup:
            inputs, labels = mixup_fn(inputs, labels)
         
        # Clear the gradients
        optimizer.zero_grad()
        
        # Forward Pass
        outputs = model(inputs)
        _, predictions = torch.max(outputs, 1)

        # Compute Loss
        loss = criterion(outputs, labels)
        
        # Calculate gradients
        loss.backward()
        
        # Update Weights
        optimizer.step()
        
        # Calculate Loss
        running_loss += loss.item() * inputs.size(0)
        if use_mixup:
            running_corrects += (predictions == torch.max(labels, 1)[1]).sum().item()
        else:
            running_corrects += (predictions == labels).sum().item()
    
    epoch_loss = running_loss / num_samples
    epoch_acc = running_corrects / num_samples
    
    return epoch_loss, epoch_acc

In [18]:
def test_epoch(model, epoch, criterion, optimizer, dataloader, device):
    model.eval()
    
    num_batches = len(dataloader)
    num_samples = len(dataloader.dataset)
    
    with torch.no_grad():
        running_loss = 0.0
        running_corrects = 0

        for batch, (inputs, labels) in enumerate(tqdm.tqdm(dataloader)):
            # Transfer Data to GPU if available
            inputs, labels = inputs.to(device), labels.to(device)
            if use_mixup:
                labels = torch.nn.functional.one_hot(labels.to(torch.int64), num_classes=2).float()

            # Clear the gradients
            optimizer.zero_grad()

            # Forward Pass
            outputs = model(inputs)
            _, predictions = torch.max(outputs, 1)

            # Compute Loss
            loss = criterion(outputs, labels)

            # Update Weights
            # optimizer.step()

            # Calculate Loss
            running_loss += loss.item() * inputs.size(0)
            if use_mixup:
                running_corrects += (predictions == torch.max(labels, 1)[1]).sum().item()
            else:
                running_corrects += (predictions == labels).sum().item()

        epoch_loss = running_loss / num_samples
        epoch_acc = running_corrects / num_samples
    
    return epoch_loss, epoch_acc

In [19]:
from torchsampler import ImbalancedDatasetSampler

batch_size = 128
epochs = 35
lr = 0.001
warmup_epochs = 5
wd = 0.01

full_data = TensorDataset(torch.Tensor(np.expand_dims(X, axis=1)), torch.from_numpy(Y))

train_size = int(0.8 * len(full_data))
test_size = len(full_data) - train_size
train_data, val_data = torch.utils.data.random_split(full_data, [train_size, test_size])

train_loader = DataLoader(train_data, batch_size=batch_size)
val_loader = DataLoader(val_data, batch_size=batch_size)

In [22]:
model = ResNet(18, Block, image_channels=1, num_classes=2)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model, device_ids=[0, 1])
model.to(device)
print(device)

cuda:0


In [23]:
import wandb

wandb.init(project="Call-NoCall", entity="frankfundel")

wandb.config = {
  "learning_rate": lr,
  "epochs": epochs,
  "batch_size": batch_size
}

criterion = nn.CrossEntropyLoss()
if use_mixup:
    criterion = nn.BCEWithLogitsLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=lr)
if use_adam:
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

scheduler = None
if use_reduceonplateu:
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

min_val_loss = np.inf

torch.autograd.set_detect_anomaly(True)
for epoch in range(epochs):
    end = time.time()
    print(f"==================== Starting at epoch {epoch} ====================", flush=True)
    
    train_loss, train_acc = train_epoch(model, epoch, criterion, optimizer, scheduler, train_loader, device)
    print('Training loss: {:.4f} Acc: {:.4f}'.format(train_loss, train_acc), flush=True)
    
    val_loss, val_acc = test_epoch(model, epoch, criterion, optimizer, val_loader, device)
    print('Validation loss: {:.4f} Acc: {:.4f}'.format(val_loss, val_acc), flush=True)
    
    if use_reduceonplateu:
        scheduler.step(val_loss)
        
    wandb.log({
        "train_loss": train_loss,
        "train_acc": train_acc,
        "val_loss": val_loss,
        "val_acc": val_acc,
    })
    
    if min_val_loss > val_loss:
        print('val_loss decreased, saving model', flush=True)
        min_val_loss = val_loss
         
        # Saving State Dict
        torch.save(model.state_dict(), 'call_nocall.pth')

train_acc,▁▅▆▇▇▇▇█▇▇▇██▇██████████████
train_loss,█▄▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁████▄██▃▇▇█████████████████
val_loss,▃▁▁▁▁▆▁▁█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,0.99899
train_loss,0.00502
val_acc,0.93159
val_loss,0.22847


==================== Starting at epoch 0 ====================


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:04<00:00,  3.52it/s]

Training loss: 0.3386 Acc: 0.8469



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 33.55it/s]

...

Validation loss: 0.2449 Acc: 0.9477
==================== Starting at epoch 34 ====================



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:03<00:00,  5.03it/s]

Training loss: 0.0041 Acc: 0.9990



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 34.18it/s]

Validation loss: 0.2452 Acc: 0.9477


In [24]:
wandb.finish()

train_acc,▁▅▆▇█▇▇▇▇██████████████████████████
train_loss,█▄▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▄████▇█▁███████████████████████████
val_loss,▂▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,0.99899
train_loss,0.00412
val_acc,0.94769
val_loss,0.24518


In [25]:
model.load_state_dict(torch.load('call_nocall.pth'))
compiled_model = torch.jit.script(model)
torch.jit.save(compiled_model, 'call_nocall.pt')

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torch.jit.load('call_nocall.pt')
model.to(device)

RecursiveScriptModule(
  original_name=ResNet
  (conv1): RecursiveScriptModule(original_name=Conv2d)
  (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
  (relu): RecursiveScriptModule(original_name=ReLU)
  (maxpool): RecursiveScriptModule(original_name=MaxPool2d)
  (layer1): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(
      original_name=Block
      (conv1): RecursiveScriptModule(original_name=Conv2d)
      (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
      (conv2): RecursiveScriptModule(original_name=Conv2d)
      (bn2): RecursiveScriptModule(original_name=BatchNorm2d)
      (conv3): RecursiveScriptModule(original_name=Conv2d)
      (bn3): RecursiveScriptModule(original_name=BatchNorm2d)
      (relu): RecursiveScriptModule(original_name=ReLU)
      (identity_downsample): RecursiveScriptModule(
        original_name=Sequential
        (0): RecursiveScriptModule(original_name=Conv2d)
        (1): RecursiveScriptModule(original_na

# Individuals

In [8]:
train_data = TensorDataset(torch.Tensor(np.expand_dims(X_train, axis=1)), torch.from_numpy(Y_train))
test_data = TensorDataset(torch.Tensor(np.expand_dims(X_test, axis=1)), torch.from_numpy(Y_test))
val_data = TensorDataset(torch.Tensor(np.expand_dims(X_val, axis=1)), torch.from_numpy(Y_val))

batch_size = 256
train_loader = DataLoader(train_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)
val_loader = DataLoader(val_data, batch_size=batch_size)
    
def callnocall(dataloader):
    preds = []
    maxes = []
    for batch, (inputs, labels) in enumerate(tqdm.tqdm(dataloader)):
        # Transfer Data to GPU if available
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, pred = torch.max(outputs, 1)
        max_ = torch.nn.functional.softmax(outputs, dim=1).tolist()
        preds.extend(pred)
        maxes.extend(max_)
            
    indices = []
    labels = []
    for i, (pred, max_) in enumerate(zip(preds, maxes)):
        if pred == 1:
            indices.append(i)
        labels.append(max_[1])
    return indices, labels

train_indices, train_labels = callnocall(train_loader)
test_indices, test_labels = callnocall(test_loader)
val_indices, val_labels = callnocall(val_loader)

with open('call_nocall.indices', 'wb') as file:
    pickle.dump([[np.asarray(train_indices), np.asarray(test_indices), np.asarray(val_indices)],
                [np.asarray(train_labels), np.asarray(test_labels), np.asarray(val_labels)]], file)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:02<00:00, 15.51it/s]


In [9]:
print(np.asarray(train_indices).shape, np.asarray(train_labels).shape)

(19839,) (31594,)


# Sequences

In [2]:
import sys
sys.path.append('../../datasets/')
import pickle
import numpy as np
import matplotlib.pyplot as plt
import h5py
from prepare_sequences import prepare, germanBats

classes = germanBats

num_bands = 257
patch_len = 44                               # = 250ms ~ 25ms
patch_skip = patch_len / 2                   # = 150ms ~ 15ms
seq_len = 60                                 # = 500ms with ~ 5 calls
seq_skip = 15


X_train, Y_train, X_test, Y_test, X_val, Y_val = prepare("../../datasets/prepared.h5", classes, patch_len, patch_skip,
                                                         seq_len, seq_skip)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:20<00:00,  1.12s/it]


In [7]:
import numpy as np
import tqdm
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

train_data = TensorDataset(torch.Tensor(np.expand_dims(X_train, axis=2)), torch.from_numpy(Y_train))
test_data = TensorDataset(torch.Tensor(np.expand_dims(X_test, axis=2)), torch.from_numpy(Y_test))
val_data = TensorDataset(torch.Tensor(np.expand_dims(X_val, axis=2)), torch.from_numpy(Y_val))

batch_size = 1
train_loader = DataLoader(train_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)
val_loader = DataLoader(val_data, batch_size=batch_size)
    
def callnocall(dataloader):
    preds = []
    maxes = []
    for batch, (inputs, labels) in enumerate(tqdm.tqdm(dataloader)):
        # Transfer Data to GPU if available
        inputs, labels = inputs[0].to(device), labels[0].to(device)
        outputs = model(inputs)
        _, pred = torch.max(outputs, 1)
        max_ = torch.nn.functional.softmax(outputs, dim=1).tolist()
        preds.append(pred) # add [60] preds
        maxes.append(max_) # add [60] maxes 
    
    # preds [19k, 60, ]
    # maxes [19k, 60, 18, ]
    
    indices = []
    labels = []
    
    for p, m in zip(preds, maxes): # for each batch e.g. sequence
        idx = []
        lbl = []
        for i, (pred, max_) in enumerate(zip(p, m)):
            if pred == 1:
                idx.append(i)
            lbl.append(max_[1])
        indices.append(idx)
        labels.append(lbl)
        
    return indices, labels

train_indices, train_labels = callnocall(train_loader)
test_indices, test_labels = callnocall(test_loader)
val_indices, val_labels = callnocall(val_loader)

with open('call_nocall_seq.indices', 'wb') as file:
    pickle.dump([[np.asarray(train_indices), np.asarray(test_indices), np.asarray(val_indices)],
                [np.asarray(train_labels), np.asarray(test_labels), np.asarray(val_labels)]], file)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2888/2888 [00:39<00:00, 72.61it/s]
/opt/Python/3.8.6/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
